In [1]:
import pandas as pd

filepath_dict = {'yelp':   'sentiment labelled sentences/yelp_labelled.txt',
                 'amazon': 'sentiment labelled sentences/amazon_cells_labelled.txt',
                 'imdb':   'sentiment labelled sentences/imdb_labelled.txt'}

In [2]:
filepath_dict

{'amazon': 'sentiment labelled sentences/amazon_cells_labelled.txt',
 'imdb': 'sentiment labelled sentences/imdb_labelled.txt',
 'yelp': 'sentiment labelled sentences/yelp_labelled.txt'}

In [3]:
df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)

In [20]:
import numpy as np
df = pd.concat(df_list)


In [43]:
var = 'source'
col_list = [var]+[cols for cols in df.columns if cols != var]


In [44]:
col_list

['source', 'sentence', 'label']

In [45]:
df = df[col_list]

In [46]:
df.head()

,source,sentence,label
0,amazon,So there is no way for me to plug it in here i...,0
1,amazon,"Good case, Excellent value.",1
2,amazon,Great for the jawbone.,1
3,amazon,Tied to charger for conversations lasting more...,0
4,amazon,The mic is great.,1


#### we  can use the CountVectorizer provided by the scikit-learn library to vectorize sentences.

In [130]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
vectorizer = CountVectorizer(min_df=0, lowercase=False)
df_amazon = df[df.source == 'amazon']

sentence_amazon = df_amazon.sentence.values
label_amazon = df_amazon.label.values

sentence_vector = vectorizer.fit_transform(sentence_amazon)
sentence_vector.shape

(1000, 2167)

In [131]:
X_train,X_test,y_train,y_test = train_test_split(sentence_vector,label_amazon,test_size=0.2,random_state=1)

In [132]:
X_train[:5]

<5x2167 sparse matrix of type '<class 'numpy.int64'>'
	with 38 stored elements in Compressed Sparse Row format>

In [133]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(800, 2167) (200, 2167)
(800,) (200,)


In [134]:
from sklearn.linear_model import LogisticRegression
classifier  = LogisticRegression()
classifier.fit(X_train,y_train)
print('Accuracy using Countvectorizer in scikitlearn  : ',classifier.score(X_test,y_test))

Accuracy using Countvectorizer in scikitlearn  :  0.775


# Using NLTK

In [136]:
import nltk
#nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer

#### CountVectorizer can lowercase letters, disregard punctuation and stopwords, but it can't LEMMATIZE or STEM


#### Initialize a CountVectorizer object: count_vectorizer
count_vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None)

#### Min_df¶
**Min_df ignores terms that have a document frequency (presence in % of documents) strictly lower than the given threshold. For example, Min_df=0.66 requires that a term appear in 66% of the docuemnts for it to be considered part of the vocabulary.

Sometimes min_df is used to limit the vocabulary size, so it learns only those terms that appear in at least 10%, 20%, etc. of the documents.


#### Max_df
When building the vocabulary, it ignores terms that have a document frequency strictly higher than the given threshold. This could be used to exclude terms that are too frequent and are unlikely to help predict the label. For example, by analyzing reviews on the movie Lion King, the term 'Lion' might appear in 90% of the reviews (documents), in which case, we could consider establishing Max_df=0.89

In [152]:
vectors = CountVectorizer()
wordnet = WordNetLemmatizer()
sentence_filter = [wordnet.lemmatize(word) for word in sentence_amazon]
sentence_vector1 =  vectors.fit_transform(sentence_filter)
sentence_vector1.shape

(1000, 1847)

In [153]:
sentence_filter[:5]

['So there is no way for me to plug it in here in the US unless I go by a converter.',
 'Good case, Excellent value.',
 'Great for the jawbone.',
 'Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!',
 'The mic is great.']

In [154]:
X_train,X_test,y_train,y_test = train_test_split(sentence_vector1,label_amazon,test_size=0.2,random_state=1)


In [155]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(800, 1847) (200, 1847)
(800,) (200,)


In [156]:
from sklearn.linear_model import LogisticRegression
classifier  = LogisticRegression()
classifier.fit(X_train,y_train)
print('Accuracy using NLTK: ',classifier.score(X_test,y_test))

Accuracy using NLTK:  0.805


# Using KERAS

In [158]:
import keras
from keras import Sequential
from keras.layers import Dense
input_dim = X_train.shape[1]  # Number of features
model = Sequential()
# Adding the input layer and the first hidden layer
model.add(Dense(activation="relu", input_dim=input_dim, units=6, kernel_initializer="uniform"))

# Adding the second hidden layer
model.add(Dense(activation="relu", units=6, kernel_initializer="uniform"))

# Adding the output layer
model.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))


# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [160]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 11088     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 11,137
Trainable params: 11,137
Non-trainable params: 0
_________________________________________________________________


In [168]:
#Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

C:\ProgramData\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/100
800/800 [==============================] - 1s 2ms/step - loss: 0.6931 - acc: 0.5113
Epoch 2/100
800/800 [==============================] - 0s 323us/step - loss: 0.6826 - acc: 0.5813
Epoch 3/100
800/800 [==============================] - 0s 343us/step - loss: 0.6025 - acc: 0.7687
Epoch 4/100
800/800 [==============================] - 0s 499us/step - loss: 0.4535 - acc: 0.9112 0s - loss: 0.4588 - acc: 0.909
Epoch 5/100
800/800 [==============================] - 0s 396us/step - loss: 0.3130 - acc: 0.9625
Epoch 6/100
800/800 [==============================] - 0s 411us/step - loss: 0.2110 - acc: 0.9825
Epoch 7/100
800/800 [==============================] - 0s 456us/step - loss: 0.1441 - acc: 0.9887
Epoch 8/100
800/800 [==============================] - 0s 619us/step - loss: 0.1025 - acc: 0.9887
Epoch 9/100
800/800 [==============================] - 0s 514us/step - loss: 0.0759 - acc: 0.9937
Epoch 10/100
800/800 [==============================] - 0s 430us/step - loss: 0.0545 - ac

800/800 [==============================] - 0s 396us/step - loss: 9.4524e-05 - acc: 1.0000
Epoch 80/100
800/800 [==============================] - 0s 495us/step - loss: 8.9739e-05 - acc: 1.0000
Epoch 81/100
800/800 [==============================] - 0s 410us/step - loss: 8.5282e-05 - acc: 1.0000
Epoch 82/100
800/800 [==============================] - 0s 294us/step - loss: 8.1045e-05 - acc: 1.0000
Epoch 83/100
800/800 [==============================] - 0s 595us/step - loss: 7.7004e-05 - acc: 1.0000
Epoch 84/100
800/800 [==============================] - 0s 525us/step - loss: 7.3140e-05 - acc: 1.0000
Epoch 85/100
800/800 [==============================] - 0s 395us/step - loss: 6.9483e-05 - acc: 1.0000
Epoch 86/100
800/800 [==============================] - 0s 356us/step - loss: 6.6093e-05 - acc: 1.0000
Epoch 87/100
800/800 [==============================] - 0s 340us/step - loss: 6.2833e-05 - acc: 1.0000
Epoch 88/100
800/800 [==============================] - 0s 316us/step - loss: 5.9728e-

In [169]:
predictions = model.predict(X_test)

In [170]:
y_pred = []
for i in predictions:
    if i > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [171]:
from sklearn.metrics import accuracy_score
print('Accuracy Score : ',accuracy_score(y_test,y_pred))

Accuracy Score :  0.83


In [172]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

Training Accuracy: 1.0000


In [173]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.8300
